In [ ]:
#Task1 
# qaoa_portfolio.py
import numpy as np
import itertools
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import QAOA, VQE
from qiskit.algorithms.optimizers import COBYLA, SLSQP
from qiskit.opflow import PauliSumOp, I, Z
from qiskit.quantum_info import Pauli
from math import comb

#build QUBO matrix
def build_qubo(Q, c, A=None, b=None, G=None, h=None, lambda_eq=100.0, mu_ineq=100.0):
    """
    Q: n x n symmetric np.ndarray (quadratic)
    c: length-n linear term
    A,b: equality constraints (A x = b)
    G,h: inequality constraints (G x <= h) -- handled as penalty G^T G
    returns: Q_qubo (n x n), q_lin (n)
    """
    n = Q.shape[0]
    Q_qubo = Q.copy()
    q_lin = c.copy()

    # equality penalties
    if A is not None and b is not None:
        # lambda * (Ax - b)^T (Ax - b) = lambda * (x^T A^T A x - 2 b^T A x + b^T b)
        AtA = A.T @ A
        Q_qubo = Q_qubo + lambda_eq * AtA
        q_lin = q_lin - 2.0 * lambda_eq * (A.T @ b)

    # inequality penalties (simple quadratic)
    if G is not None and h is not None:
        GtG = G.T @ G
        Q_qubo = Q_qubo + mu_ineq * GtG
        q_lin = q_lin - 2.0 * mu_ineq * (G.T @ h)

    # Return QUBO format: matrix and linear vector (we will compute x^T Q x + q^T x)
    return Q_qubo, q_lin


# Objective evaluation
def qubo_obj(x, Q_qubo, q_lin):
    x = np.array(x).reshape(-1)
    return float(x @ Q_qubo @ x + q_lin @ x)


# Brute-force classical baseline (only for small n)
def brute_force_best(Q_qubo, q_lin):
    n = Q_qubo.shape[0]
    best_val = None
    best_x = None
    for bits in itertools.product([0,1], repeat=n):
        val = qubo_obj(bits, Q_qubo, q_lin)
        if best_val is None or val < best_val:
            best_val = val
            best_x = np.array(bits)
    return best_val, best_x


# Convert QUBO to PauliSumOp (simple diagonal mapping)
def qubo_to_z_op(Q_qubo, q_lin):
    """
    Construct a diagonal operator representing the QUBO objective
    as a sum of Z Pauli strings using the x=(1-Z)/2 mapping.
    This is suitable for small n demonstration.
    """
    n = Q_qubo.shape[0]
    # Expand QUBO into coefficients on products of (1-Z_i)/2 terms
    # We will create PauliSumOp op such that <x|op|x> = qubo_obj(x)
    from qiskit.opflow import SummedOp, PauliOp
    ops = []
    const = 0.0

    # Linear and quadratic contributions via expansions
    for i in range(n):
        # x_i = (1 - Z_i)/2
        # linear term q_lin[i] * x_i -> q_lin[i]/2 * (1) + (-q_lin[i]/2)*Z_i
        const += q_lin[i] * 0.5
        zi_coeff = -0.5 * q_lin[i]
        pauli = Pauli(( 'I'*n ).replace('I', 'I'))  # placeholder

    # For readability and correctness, we'll compute operator by enumerating computational basis (for small n)
    # Build diagonal vector
    diag = np.zeros(2**n)
    for state in range(2**n):
        x = np.array(list(map(int, bin(state)[2:].zfill(n))))
        diag[state] = qubo_obj(x, Q_qubo, q_lin)
    # Convert diagonal to PauliSumOp by writing it as sum_{z} coeff_z |z><z| ; Qiskit can accept as matrix op
    from qiskit.opflow import MatrixOp
    mat = np.diag(diag)
    op = MatrixOp(mat)
    return op


# QAOA run (small n)
def run_qaoa(Q_qubo, q_lin, p=1, seed=42):
    n = Q_qubo.shape[0]
    backend = Aer.get_backend('aer_simulator_statevector')
    qi = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed)

    # Build cost operator
    cost_op = qubo_to_z_op(Q_qubo, q_lin)

    optimizer = COBYLA(maxiter=200)
    qaoa = QAOA(reps=p, optimizer=optimizer, quantum_instance=qi)

    result = qaoa.compute_minimum_eigenvalue(operator=cost_op)
    # result.eigenstate is vector; get most probable bitstring
    statevec = np.asarray(result.eigenstate)
    probs = np.abs(statevec)**2
    best_state_idx = int(np.argmax(probs))
    x = np.array(list(map(int, bin(best_state_idx)[2:].zfill(n))))
    val = qubo_obj(x, Q_qubo, q_lin)
    return {'x': x, 'value': val, 'raw': result}


# Example usage
if __name__ == "__main__":
    # Toy example: 4 assets
    n = 4
    # Construct Q (symmetric) and c
    # Here Q ~ covariance-like, c ~ negative expected returns (we minimize)
    rng = np.random.RandomState(1)
    M = rng.randn(n, n)
    Q = (M + M.T) * 0.5
    c = -0.2 * rng.randn(n)

    # Simple equality constraint: pick exactly k assets (k-of-n constraint)
    k = 2
    A = np.ones((1, n))
    b = np.array([k])

    Q_qubo, q_lin = build_qubo(Q, c, A=A, b=b, lambda_eq=50.0)

    # Classical baseline
    best_val, best_x = brute_force_best(Q_qubo, q_lin)
    print("Classical brute best:", best_val, best_x)

    # QAOA attempt
    qaoa_res = run_qaoa(Q_qubo, q_lin, p=1)
    print("QAOA result:", qaoa_res['value'], qaoa_res['x'])


In [ ]:
#Task2: Write a quantum optimization program
from qiskit.optimization import QuadraticProgram
from qiskit.algorithms import MinimumEigenOptimizer
from qiskit.algorithms import NumPyMinimumEigensolver

def build_quadratic_program(weights, pmv, target_pmv):
    qp = QuadraticProgram()
    n = len(weights)
    
    # Add binary variables
    for i in range(n):
        qp.binary_var(name=f'x{i}')

    # Objective: maximize weights
    qp.maximize(linear=weights)

    # Budget constraint penalty
    qp.minimize(0.1 * (qp.sum([pmv[i] * qp.variables[i] for i in range(n)]) - target_pmv) ** 2)
    
    return qp

def run_qiskit_optimizer(qp):
    optimizer = MinimumEigenOptimizer(NumPyMinimumEigensolver())
    result = optimizer.solve(qp)
    return result

# Example usage
pmv = np.random.rand(n) * 1000  # Example market values
target_pmv = 5000  # Target portfolio market value
qp = build_quadratic_program(weights, pmv, target_pmv)
optimizer_result = run_qiskit_optimizer(qp)

In [ ]:
#Task3: Solve the optimization problem using the quantum formulation
class ETFOptimizer:
    def __init__(self, assets):
        self.assets = assets

    def bucketize(self):
        # Implement logic to group assets based on characteristics
        pass

    def meta_optimize(self):
        # Combine QAOA and classical optimizers
        pass

    def handle_constraints(self, constraints):
        # Implement logic to enforce constraints
        pass

# Example usage
assets = generate_random_portfolio(10)
optimizer = ETFOptimizer(assets)
optimizer.bucketize()

In [ ]:
def penalty_function(weights, risk_threshold):
    risk = calculate_risk(weights)  # Define this function to compute risk
    if risk > risk_threshold:
        return risk - risk_threshold
    return 0

def run_with_penalties(weights, risk_threshold):
    penalty = penalty_function(weights, risk_threshold)
    # Incorporate penalty into optimization objective
    return penalty

def validate_portfolio(result, benchmark):
    # Implement validation logic to compare results against benchmarks
    pass

# Example usage
risk_threshold = 0.1  # Example risk threshold
penalty = run_with_penalties(weights, risk_threshold)

In [ ]:
#Validate vs a classical solver
def run_experiments(num_trials):
    for _ in range(num_trials):
        weights = generate_random_portfolio(n)
        qp = build_quadratic_program(weights, pmv, target_pmv)
        optimizer_result = run_qiskit_optimizer(qp)
        validate_portfolio(optimizer_result, benchmark)

# Example usage
num_trials = 10  # Number of trials to run
run_experiments(num_trials)
